In [1]:
from _element import feature_control as ft_c

In [2]:
from _usecase import algorithm_prophet as prpt

In [3]:
from _element import calculations as calc

In [4]:
import pandas as pd

In [5]:
from datetime import datetime

In [6]:
from _element import varr

In [28]:
INPUT_FILENAME= 'KPP일별투입(10_17)_raw.xlsx'

In [ ]:
df_raw= ft_c.xlsx_opener(varr.PJ_DIR+varr.DF_DIR, INPUT_FILENAME)

In [30]:
df_temp= ft_c.xlsx_opener(varr.PJ_DIR+varr.TEMP_DATA_DIR, 'temp_data_merged.xlsx')

In [31]:
y_col= df_raw.columns.values.tolist()

In [32]:
try:
    y_col.remove('ds')
except ValueError:
    pass

In [33]:
y_col

['y']

In [34]:
x_col= df_temp.columns.values.tolist()

In [35]:
x_col

['ds', 'temp_max', 'temp_min', 'rain_amount']

In [36]:
df_raw['ds']= pd.to_datetime(df_raw['ds'], box=True, format= '%Y/%m/%d', exact=True)

In [37]:
df_temp['ds']= pd.to_datetime(df_temp['ds'], box=True, format= '%Y/%m/%d', exact=True)

In [38]:
df_raw= pd.merge(df_raw, df_temp, how='inner', on='ds')

In [39]:
df_raw.head()

,ds,y,temp_max,temp_min,rain_amount
0,2010-07-01,79590,30.500000,24.900000,0.0
1,2010-07-02,79456,25.799999,22.700001,70.0
2,2010-07-03,48469,25.600000,22.600000,1.0
3,2010-07-04,1045,29.700001,23.100000,0.0
4,2010-07-05,65049,30.600000,21.799999,0.0


In [46]:
df_weekday= {}
split_point= 7
for i in range(split_point):
    df_splited= df_raw.loc[df_raw.ds.apply(lambda x: x.weekday())==i]
    df_weekday[i]= {}
    for y in y_col:
        df_weekday[i][y]= ft_c.train_test_sample(
            df_splited, y, x_col, forecastday= varr.FORECASTDAY//split_point)

In [47]:
df_weekday[0]['y']

{'test':              ds       y  temp_max  temp_min  rain_amount
 2699 2017-11-27  117480       8.1       1.8          0.0,
 'testX':              ds  temp_max  temp_min  rain_amount
 2699 2017-11-27       8.1       1.8          0.0,
 'testY': 2699    117480
 Name: y, dtype: int64,
 'train':              ds       y   temp_max   temp_min  rain_amount
 4    2010-07-05   65049  30.600000  21.799999          0.0
 11   2010-07-12   67680  28.299999  22.200001          0.0
 18   2010-07-19   73816  29.299999  24.200001          0.0
 25   2010-07-26   75731  27.600000  23.400000          6.0
 32   2010-08-02   67290  31.600000  24.000000          1.0
 39   2010-08-09   67691  32.799999  26.400000          0.0
 46   2010-08-16   70947  26.500000  21.000000          0.0
 53   2010-08-23   71422  28.900000  23.500000         13.0
 60   2010-08-30   68075  29.799999  22.299999          0.0
 67   2010-09-06   74425  29.700001  23.000000          0.0
 74   2010-09-13   75663  27.400000  21.100000 

In [48]:
result_dict= {}
for y in y_col:
    result_dict[y]= {
                    'mon': prpt.Bayseian2(df_weekday[0][y], 'week'),\
                    'tue': prpt.Bayseian2(df_weekday[1][y], 'week'),\
                    'wed': prpt.Bayseian2(df_weekday[2][y], 'week'),\
                    'thu': prpt.Bayseian2(df_weekday[3][y], 'week'),\
                    'pri': prpt.Bayseian2(df_weekday[4][y], 'week'),\
                    'sat': prpt.Bayseian2(df_weekday[5][y], 'week'),\
                    'sun': prpt.Bayseian2(df_weekday[6][y], 'week'),\
                    }

In [49]:
merged_dict={}
for y in y_col:
    merged_dict[y]={'future': 
                    pd.concat([result_dict[y]['mon']['future'],\
                              result_dict[y]['tue']['future'],\
                              result_dict[y]['wed']['future'],\
                              result_dict[y]['thu']['future'],\
                              result_dict[y]['pri']['future'],\
                              result_dict[y]['sat']['future'],\
                              result_dict[y]['sun']['future']],\
                              axis= 0).sort_values(by='ds', axis=0),
                   'forecastProphetTable':
                    pd.concat([result_dict[y]['mon']['forecastProphetTable'],\
                              result_dict[y]['tue']['forecastProphetTable'],\
                              result_dict[y]['wed']['forecastProphetTable'],\
                              result_dict[y]['thu']['forecastProphetTable'],\
                              result_dict[y]['pri']['forecastProphetTable'],\
                              result_dict[y]['sat']['forecastProphetTable'],\
                              result_dict[y]['sun']['forecastProphetTable']],\
                              axis= 0).sort_values(by='ds', axis=0)
                   }

In [50]:
forecast_dict={}
print(INPUT_FILENAME)
for y in y_col:
    merged_dict[y]['future']['ds']= pd.to_datetime(merged_dict[y]['future']['ds'], box=True, format= '%Y/%m/%d', exact=True)
    merged_dict[y]['forecastProphetTable']['ds']= pd.to_datetime(merged_dict[y]['forecastProphetTable']['ds'], box=True, format= '%Y/%m/%d', exact=True)
    forecast_dict[y]= prpt.extract_info_from(merged_dict[y]['future'], merged_dict[y]['forecastProphetTable'], varr.FORECASTDAY)
    forecast_dict[y]['result_df']= pd.merge(forecast_dict[y]['result_df'], df_raw[['ds', y]], how='inner', on='ds')
    print('RMSE with segment '+str(y)+': '+str(calc.rms_error(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print('MAPE with segment '+str(y)+': '+str(calc.map_error(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print('MAPE_with_std with segment '+str(y)+': '+str(calc.map_error_with_std(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print('MAPE_div_std with segment '+str(y)+': '+str(calc.map_error_div_std(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print('sMAPE of company code '+str(y)+': '+str(calc.smap_error(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print(forecast_dict[y]['result_df'].head())
    ft_c.save_as_xlsx(varr.PJ_DIR+varr.DF_DIR, forecast_dict[y]['result_df'], INPUT_FILENAME, '_seg'+str(y)+'withweekday')
    print('\n')

KPP일별투입(10_17)_raw.xlsx
RMSE with segment y: 6476.52747853
MAPE with segment y: 45.6762554537
MAPE_with_std with segment y: 20.6395919102
MAPE_div_std with segment y: 73.5508237695
sMAPE of company code y: 4.87943060612
          ds  temp_max  temp_min  rain_amount           yhat       y
0 2017-11-24       2.9      -2.5          0.1  125038.754153  120647
1 2017-11-25       6.4       0.8         15.7   62623.574508   67756
2 2017-11-26       6.4      -2.4          0.0    -188.359619     100
3 2017-11-27       8.1       1.8          0.0  112834.287908  117480
4 2017-11-28       9.8       4.2          1.2  121613.045764  122192




In [23]:
# result_dict[y]['sun']['model'].plot(result_dict[y]['sun']['forecastProphetTable'])

In [24]:
# result_dict[y]['sun']['model'].plot_components(result_dict[y]['sun']['forecastProphetTable'])